In [6]:
import math
import torch.nn as nn
import argparse
import random
import warnings
import numpy as np
import torch
import torch.nn.functional as F
from torch import optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable
import itertools
import pandas as pd
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math

seed = 7
torch.manual_seed(seed)
np.random.seed(seed)


In [2]:
pfamA_motors = pd.read_csv("../../data/pfamA_motors.csv")
df_dev = pd.read_csv("../../data/df_dev.csv")
motor_toolkit = pd.read_csv("../../data/motor_tookits.csv")

pfamA_motors_balanced = pfamA_motors.groupby('clan').apply(lambda _df: _df.sample(4500,random_state=1))
pfamA_motors_balanced = pfamA_motors_balanced.apply(lambda x: x.reset_index(drop = True))

pfamA_target_name = ["PF00349","PF00022","PF03727","PF06723",\
                       "PF14450","PF03953","PF12327","PF00091","PF10644",\
                      "PF13809","PF14881","PF00063","PF00225","PF03028"]

pfamA_target = pfamA_motors.loc[pfamA_motors["pfamA_acc"].isin(pfamA_target_name),:]


# shuffle pfamA_target and pfamA_motors_balanced
pfamA_target = pfamA_target.sample(frac = 1)
pfamA_target_ind = pfamA_target.iloc[:,0]
print(pfamA_target_ind[0:5])
print(pfamA_motors_balanced.shape)

pfamA_motors_balanced = pfamA_motors_balanced.sample(frac = 1) 
pfamA_motors_balanced_ind = pfamA_motors_balanced.iloc[:,0]
print(pfamA_motors_balanced_ind[0:5])
print(pfamA_target.shape)



179519      179519
1414859    1414859
12920        12920
1415258    1415258
13385        13385
Name: Unnamed: 0, dtype: int64
(18000, 6)
13493    180756
1539     166414
2688     131988
1691      37094
188      130155
Name: Unnamed: 0, dtype: int64
(59149, 6)


In [3]:
aminoacid_list = [
    'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
    'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'
]
clan_list = ["actin_like","tubulin_c","tubulin_binding","p_loop_gtpase"]
        
aa_to_ix = dict(zip(aminoacid_list, np.arange(1, 21)))
clan_to_ix = dict(zip(clan_list, np.arange(0, 4)))

def word_to_index(seq,to_ix):
    "Returns a list of indices (integers) from a list of words."
    return [to_ix.get(word, 0) for word in seq]

ix_to_aa = dict(zip(np.arange(1, 21), aminoacid_list))
ix_to_clan = dict(zip(np.arange(0, 4), clan_list))

def index_to_word(ixs,ix_to): 
    "Returns a list of words, given a list of their corresponding indices."
    return [ix_to.get(ix, 'X') for ix in ixs]

def prepare_sequence(seq):
    idxs = word_to_index(seq[0:-1],aa_to_ix)
    return torch.tensor(idxs, dtype=torch.long)

def prepare_labels(seq):
    idxs = word_to_index(seq[1:],aa_to_ix)
    return torch.tensor(idxs, dtype=torch.long)

def prepare_eval(seq):
    idxs = word_to_index(seq[:],aa_to_ix)
    return torch.tensor(idxs, dtype=torch.long)

prepare_labels('YCHXXXXX')

tensor([2, 7, 0, 0, 0, 0, 0])

In [4]:
# set device
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
class PositionalEncoding(nn.Module):
    """
    PositionalEncoding module injects some information about the relative or absolute position of
    the tokens in the sequence. The positional encodings have the same dimension as the embeddings 
    so that the two can be summed. Here, we use sine and cosine functions of different frequencies.
    """
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)

    def forward(self, x):
#         x = x + self.pe[:x.size(0), :]
#         print("x.size() : ", x.size())
#         print("self.pe.size() :", self.pe[:x.size(0),:,:].size())
        x = torch.add(x ,Variable(self.pe[:x.size(0),:,:], requires_grad=False))
        return self.dropout(x)

    
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != src.size(0):
            device = src.device
            mask = self._generate_square_subsequent_mask(src.size(0)).to(device)
            self.src_mask = mask
#         print("src.device: ", src.device)
        src = self.encoder(src) * math.sqrt(self.ninp)
#         print("self.encoder(src) size: ", src.size())
        src = self.pos_encoder(src)
#         print("elf.pos_encoder(src) size: ", src.size())
        output = self.transformer_encoder(src, self.src_mask)
#         print("output size: ", output.size())
        output = self.decoder(output)
        return output
    
    
ntokens = len(aminoacid_list) + 1 # the size of vocabulary
emsize = 12 # embedding dimension
nhid = 100 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 6 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 12 # the number of heads in the multiheadattention models
dropout = 0.1 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)



In [8]:
criterion = nn.CrossEntropyLoss()
lr = 3.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)


In [9]:
# load the trained weights on df_dev
model.load_state_dict(torch.load("../../data/transformer_encoder_201025.pt"))

<All keys matched successfully>

In [10]:
model.to(device)
model.train() # Turn on the train mode

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=12, out_features=12, bias=True)
        )
        (linear1): Linear(in_features=12, out_features=100, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=100, out_features=12, bias=True)
        (norm1): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=12, out_features=12, bias=True)
        )
        (linear1): Linear(in_features=12, out_features=100, bias=T

## Proceed weight updates using motor_balanced

In [13]:
import time

In [14]:
start_time = time.time()
print_every = 1000
# loss_vector = []

for epoch in np.arange(0, pfamA_motors_balanced.shape[0]): 
    seq = pfamA_motors_balanced.iloc[epoch, 3]
    
    sentence_in = prepare_sequence(seq)
    targets = prepare_labels(seq)
#     sentence_in = sentence_in.to(device = device)
    sentence_in = sentence_in.unsqueeze(1).to(device = device)
    targets = targets.to(device = device)
    
    optimizer.zero_grad()
    output = model(sentence_in)
    
#     print("targets size: ", targets.size())
    loss = criterion(output.view(-1, ntokens), targets)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    if epoch % print_every == 0:
        print(f"At Epoch: %.1f"% epoch)
        print(f"Loss %.4f"% loss)
        elapsed = time.time() - start_time
        print(f"time elapsed %.4f"% elapsed)
#         torch.save(model.state_dict(), "../../data/transformer_encoder_201025.pt")
#     loss_vector.append(loss)


At Epoch: 0.0
Loss 2.9313
time elapsed 0.0462
At Epoch: 1000.0
Loss 2.9778
time elapsed 19.2431
At Epoch: 2000.0
Loss 2.8527
time elapsed 38.5000
At Epoch: 3000.0
Loss 2.7362
time elapsed 57.9419
At Epoch: 4000.0
Loss 2.7080
time elapsed 77.5878
At Epoch: 5000.0
Loss 2.8469
time elapsed 97.5463
At Epoch: 6000.0
Loss 2.8770
time elapsed 117.0020
At Epoch: 7000.0
Loss 2.8420
time elapsed 136.6154
At Epoch: 8000.0
Loss 2.8524
time elapsed 156.3122
At Epoch: 9000.0
Loss 2.7866
time elapsed 175.9095
At Epoch: 10000.0
Loss 2.8042
time elapsed 199.8341
At Epoch: 11000.0
Loss 2.7702
time elapsed 238.7344
At Epoch: 12000.0
Loss 2.9439
time elapsed 283.1732
At Epoch: 13000.0
Loss 2.7778
time elapsed 327.0251
At Epoch: 14000.0
Loss 2.8029
time elapsed 371.7587
At Epoch: 15000.0
Loss 2.8372
time elapsed 417.0790
At Epoch: 16000.0
Loss 2.8199
time elapsed 463.3373
At Epoch: 17000.0
Loss 2.9194
time elapsed 510.2089


In [15]:
torch.save(model.state_dict(), "../../data/evotune_transformerencoder_balanced.pt")

In [16]:
print("done")

done


## Proceed weight updates using the entire pfam_motor set

In [18]:
start_time = time.time()
print_every = 1000
# loss_vector = []

for epoch in np.arange(0, pfamA_target.shape[0]): 
    seq = pfamA_target.iloc[epoch, 3]
    
    sentence_in = prepare_sequence(seq)
    targets = prepare_labels(seq)
#     sentence_in = sentence_in.to(device = device)
    sentence_in = sentence_in.unsqueeze(1).to(device = device)
    targets = targets.to(device = device)
    
    optimizer.zero_grad()
    output = model(sentence_in)
    
#     print("targets size: ", targets.size())
    loss = criterion(output.view(-1, ntokens), targets)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    if epoch % print_every == 0:
        print(f"At Epoch: %.1f"% epoch)
        print(f"Loss %.4f"% loss)
        elapsed = time.time() - start_time
        print(f"time elapsed %.4f"% elapsed)
#         torch.save(model.state_dict(), "../../data/transformer_encoder_201025.pt")
#     loss_vector.append(loss)


At Epoch: 0.0
Loss 2.8420
time elapsed 0.0300
At Epoch: 1000.0
Loss 2.9051
time elapsed 53.5168
At Epoch: 2000.0
Loss 2.8652
time elapsed 110.0196
At Epoch: 3000.0
Loss 2.9730
time elapsed 164.7915
At Epoch: 4000.0
Loss 2.8525
time elapsed 220.6136
At Epoch: 5000.0
Loss 2.8726
time elapsed 275.8973
At Epoch: 6000.0
Loss 2.8937
time elapsed 328.3026
At Epoch: 7000.0
Loss 2.9131
time elapsed 381.6925
At Epoch: 8000.0
Loss 2.9119
time elapsed 433.0885
At Epoch: 9000.0
Loss 2.9055
time elapsed 486.7597
At Epoch: 10000.0
Loss 2.9505
time elapsed 542.0930
At Epoch: 11000.0
Loss 2.8767
time elapsed 596.7346
At Epoch: 12000.0
Loss 2.9729
time elapsed 651.6664
At Epoch: 13000.0
Loss 2.8220
time elapsed 704.6541
At Epoch: 14000.0
Loss 2.8093
time elapsed 758.7925
At Epoch: 15000.0
Loss 2.8726
time elapsed 780.7511
At Epoch: 16000.0
Loss 2.9226
time elapsed 799.7534
At Epoch: 17000.0
Loss 2.7995
time elapsed 818.7104
At Epoch: 18000.0
Loss 2.8208
time elapsed 837.6963
At Epoch: 19000.0
Loss 2.899

In [19]:
torch.save(model.state_dict(), "../../data/evotune_transformerencoder_balanced_target.pt")

In [20]:
print("done")

done
